# Stock Volatility Prediction (Sample for Pr. Collado)
### in this project , I will predict stock volatility by using 
### GARCH model
### Linear model
### non-linear model

In [1]:
from bokeh.io import output_notebook
output_notebook()

Loading BokehJS ...

In [2]:
import pandas as pd
import numpy as np
from arch import arch_model 
from sklearn.metrics import r2_score
from stock_data_reader import StockDataReader
reader = StockDataReader('7az7vgtTPxiyj2Ncsc-H')

In [3]:
from IPython.core.display import display, HTML
display(HTML("<style>.container { width:80% !important; }</style>"))

## Construct dataset
### Construct dataset for models

In [4]:
# Create a dataframe for a simple linear regression, this part is only for the spark sample
# yesterday's vol, yesterday's return
def linear_data(ticker,window,thresh_date,export=False):
    reader.initialize_data(ticker,'2005-01-01','2016-12-31')
    dat = reader.price_table
    dat = (dat.set_index(dat['date'])).drop(['date','adj_volume'],axis=1)
    dat['return'] = (np.log(dat['adj_close'])-np.log((dat['adj_close']).shift(1)))*100

    sigma_column_name = str(window)+'dayvar'
    dat[sigma_column_name]= dat['return'].rolling(window,center=False).var()
    if export==True:
        return dat
    dat_yesterday = dat[['return',sigma_column_name]].shift(1)
    dat_yesterday_column_name = [x+'_yesterday' for x in list(dat_yesterday.columns)]
    dat_yesterday.columns = dat_yesterday_column_name
    data_linear = (pd.concat([dat,dat_yesterday],
                            axis=1)).drop(['adj_close','return'],axis=1)
    return data_linear[thresh_date:]


In [5]:
# Create dataset for other high-dimension models
def get_data(ticker,time_window,sigma_window,thresh_date):
    dat=linear_data(ticker,sigma_window,'Null',True)
    sigma_column_name = str(sigma_window)+'dayvar'
    dat_shift = dat[['return',sigma_column_name]]
    columns = list(dat_shift.columns)
    dat_copy = dat.copy()
    for day in range(1,time_window+1):
        dat_tmp = dat_shift.shift(day)
        dat_tmp.columns = [name+'_'+str(day)+'daybefore' for name in columns]
        dat_copy = pd.concat([dat_copy,dat_tmp],axis=1)
    data_rnn = (dat_copy.drop(['adj_close','return'],axis=1))[thresh_date:]
    return data_rnn

## Run models

### This section is done by spark, by pyspark.ml and pyspark dataframe, I can handle dataset larger than memory. ( can also done by tensorflow if there is a pc with good GPU) .  Due to the size of this dataset, there is no need to use spark. This is just a little sample about machine learning by spark. In short, this is a sample about how to use spark building a model to predict volatility

In [6]:
from pyspark.sql import SparkSession
spark = SparkSession.builder.master("local").appName("stock_data_builder").getOrCreate()

In [7]:
# create a pyspark dataframe
ticker = 'AAPL'
sigma_window = 21
thresh_date = '2009-01-01'
dat = linear_data(ticker,sigma_window,thresh_date)
train_spark = dat[:'2016-01-01']
test_spark = dat['2016-01-01':]
train_df = spark.createDataFrame(train_spark)
test_df = spark.createDataFrame(test_spark)

In [8]:
# transform the original pyspark dataset into dense one,
# create 'features' to feed pyspark model
from pyspark.ml.feature import VectorAssembler
vec_transfer = VectorAssembler(inputCols=['return_yesterday','21dayvar_yesterday'],
                               outputCol='features')
train_spark = vec_transfer.transform(train_df).select('21dayvar','features')
test_spark = vec_transfer.transform(test_df).select('21dayvar','features')

In [9]:
#train and predict model
from pyspark.ml.regression import LinearRegression
lr = LinearRegression(labelCol='21dayvar')
model = lr.fit(train_spark)
prediction_train = model.transform(train_spark)
prediction_test = model.transform(test_spark)

In [10]:
from pyspark.ml.evaluation import RegressionEvaluator
evaluator = RegressionEvaluator(labelCol="21dayvar", predictionCol="prediction",
                                metricName="r2")
in_sample = evaluator.evaluate(prediction_train)
out_sample = evaluator.evaluate(prediction_test)
print('in sample r-square:%f'%(in_sample))
print('out of sample r-square:%f'%(out_sample))

in sample r-square:0.954956
out of sample r-square:0.944907


## From this section, I will run : GARCH 1 1, linear regression, Random Forest, RNN

In [11]:
class data_split(object):
    """this class is for train-validation-testing split"""
    def __init__(self):
        print('return form: y_train,x_train,(y_vali,x_vail),y_test,x_test')
        pass
    def train_test_split(self,dat,split_date):
        """I didn't put a validation set here because:
        1: there is no need to tune hyper parameter
        2: dataset is small ,I want to use more data
        """
        train = dat[:split_date]
        test  = dat[split_date:]
        return train.iloc[:,0],train.iloc[:,1:],test.iloc[:,0],test.iloc[:,1:]
    def train_validate_test_split(self,dat,split1,split2):
        train = dat[:split1]
        validate = dat[split1:split2]
        test = dat[split2:]
        return train.iloc[:,0],train.iloc[:,1:],validate.iloc[:,0],validate.iloc[:,1:],test.iloc[:,0],test.iloc[:,1:]

In [12]:
split = data_split()

return form: y_train,x_train,(y_vali,x_vail),y_test,x_test


### GARCH(1,1) model

In [13]:
def Garch(ticker,p_=1,q_=1):
    reader.initialize_data('AAPL','2006-01-01','2016-12-31')
    price_dat = reader.price_table
    ret = (np.log(price_dat['adj_close'])-np.log((price_dat['adj_close']).shift(1)))*100
    price_dat['returns']=ret
    price_dat = (price_dat.set_index(price_dat['date']))['returns']
    train_garch = price_dat['2009-01-01':'2016-01-01']
    am = arch_model(train_garch, vol='Garch', p=p_, o=0, q=q_, dist='Normal')
    res = am.fit(update_freq=5)
    w = res.params['omega']
    a = res.params['alpha[1]']
    b = res.params['beta[1]']
    dat_test=linear_data(ticker,21,'2006-01-01','2016-12-31')
    for i in range(1,501):# here i created 500 paths for random walks, for every path I calculated the predicted var, then take average of them
        dat_test['predict_var_%d'%(i)]=(a*((np.random.randn(len(dat_test)))**2)+b)*dat_test['21dayvar_yesterday']
    predict = (dat_test.iloc[:,3:]).mean(axis=1)
    r2 = r2_score(dat_test['21dayvar'],predict)
    return r2

In [14]:
r2_garch = Garch('AAPL')
print('r-square for garch(11) is %f'%r2_garch)

Iteration:      5,   Func. Count:     38,   Neg. LLF: 3424.5936300314956
Iteration:     10,   Func. Count:     72,   Neg. LLF: 3423.209982260805
Optimization terminated successfully.    (Exit mode 0)
            Current function value: 3423.2095279109335
            Iterations: 12
            Function evaluations: 85
            Gradient evaluations: 12
r-square for garch(11) is 0.973674


### Random Forest

In [15]:
from sklearn.ensemble import RandomForestRegressor
rf = RandomForestRegressor()

In [16]:
ticker = 'AAPL'
dat_rf = get_data(ticker,10,21,'2009-01-01')
y_train,x_train,y_test,x_test = split.train_test_split(dat_rf,'2016-01-01')
rf.fit(x_train,y_train)
rf.score(x_train,y_train)
rf.score(x_test,y_test)

RandomForestRegressor(bootstrap=True, criterion='mse', max_depth=None,
           max_features='auto', max_leaf_nodes=None,
           min_impurity_split=1e-07, min_samples_leaf=1,
           min_samples_split=2, min_weight_fraction_leaf=0.0,
           n_estimators=10, n_jobs=1, oob_score=False, random_state=None,
           verbose=0, warm_start=False)

0.99092591406392405

0.93185686031587922

### From the result we can find , it is a little bit over-fitted, so I will tune the model based on validation set

In [17]:
y_train,x_train,y_vali,x_vali,y_test,x_test = split.train_validate_test_split(dat_rf,'2015-01-01','2016-01-01')

In [18]:
depth = range(3,20)
tree_num = 20 # add more trees to reduce variance
result = []
for num in depth:
    rf_v = RandomForestRegressor(max_depth=num,n_estimators=tree_num)
    rf_v.fit(x_train,y_train)
    result.append([rf_v.score(x_train,y_train),rf_v.score(x_vali,y_vali)])

RandomForestRegressor(bootstrap=True, criterion='mse', max_depth=3,
           max_features='auto', max_leaf_nodes=None,
           min_impurity_split=1e-07, min_samples_leaf=1,
           min_samples_split=2, min_weight_fraction_leaf=0.0,
           n_estimators=20, n_jobs=1, oob_score=False, random_state=None,
           verbose=0, warm_start=False)

RandomForestRegressor(bootstrap=True, criterion='mse', max_depth=4,
           max_features='auto', max_leaf_nodes=None,
           min_impurity_split=1e-07, min_samples_leaf=1,
           min_samples_split=2, min_weight_fraction_leaf=0.0,
           n_estimators=20, n_jobs=1, oob_score=False, random_state=None,
           verbose=0, warm_start=False)

RandomForestRegressor(bootstrap=True, criterion='mse', max_depth=5,
           max_features='auto', max_leaf_nodes=None,
           min_impurity_split=1e-07, min_samples_leaf=1,
           min_samples_split=2, min_weight_fraction_leaf=0.0,
           n_estimators=20, n_jobs=1, oob_score=False, random_state=None,
           verbose=0, warm_start=False)

RandomForestRegressor(bootstrap=True, criterion='mse', max_depth=6,
           max_features='auto', max_leaf_nodes=None,
           min_impurity_split=1e-07, min_samples_leaf=1,
           min_samples_split=2, min_weight_fraction_leaf=0.0,
           n_estimators=20, n_jobs=1, oob_score=False, random_state=None,
           verbose=0, warm_start=False)

RandomForestRegressor(bootstrap=True, criterion='mse', max_depth=7,
           max_features='auto', max_leaf_nodes=None,
           min_impurity_split=1e-07, min_samples_leaf=1,
           min_samples_split=2, min_weight_fraction_leaf=0.0,
           n_estimators=20, n_jobs=1, oob_score=False, random_state=None,
           verbose=0, warm_start=False)

...

RandomForestRegressor(bootstrap=True, criterion='mse', max_depth=15,
           max_features='auto', max_leaf_nodes=None,
           min_impurity_split=1e-07, min_samples_leaf=1,
           min_samples_split=2, min_weight_fraction_leaf=0.0,
           n_estimators=20, n_jobs=1, oob_score=False, random_state=None,
           verbose=0, warm_start=False)

RandomForestRegressor(bootstrap=True, criterion='mse', max_depth=16,
           max_features='auto', max_leaf_nodes=None,
           min_impurity_split=1e-07, min_samples_leaf=1,
           min_samples_split=2, min_weight_fraction_leaf=0.0,
           n_estimators=20, n_jobs=1, oob_score=False, random_state=None,
           verbose=0, warm_start=False)

RandomForestRegressor(bootstrap=True, criterion='mse', max_depth=17,
           max_features='auto', max_leaf_nodes=None,
           min_impurity_split=1e-07, min_samples_leaf=1,
           min_samples_split=2, min_weight_fraction_leaf=0.0,
           n_estimators=20, n_jobs=1, oob_score=False, random_state=None,
           verbose=0, warm_start=False)

RandomForestRegressor(bootstrap=True, criterion='mse', max_depth=18,
           max_features='auto', max_leaf_nodes=None,
           min_impurity_split=1e-07, min_samples_leaf=1,
           min_samples_split=2, min_weight_fraction_leaf=0.0,
           n_estimators=20, n_jobs=1, oob_score=False, random_state=None,
           verbose=0, warm_start=False)

RandomForestRegressor(bootstrap=True, criterion='mse', max_depth=19,
           max_features='auto', max_leaf_nodes=None,
           min_impurity_split=1e-07, min_samples_leaf=1,
           min_samples_split=2, min_weight_fraction_leaf=0.0,
           n_estimators=20, n_jobs=1, oob_score=False, random_state=None,
           verbose=0, warm_start=False)

In [19]:
from bokeh.plotting import figure,show
plot = figure(plot_width=600,plot_height=300)

In [20]:
insample = [x[0] for x in result]
vali = [x[1] for x in result]

In [21]:
x_axis = range(3,20)
plot.line(x_axis,insample,color='red')
plot.line(x_axis,vali,color='green')
show(plot)

GlyphRenderer(id='75123ba7-3a22-473c-b8c4-87331cd7c119', ...)

GlyphRenderer(id='50a8d86d-2058-4b33-a336-d89c880a3d29', ...)

### from the figure we can see , max_depth=5 or 15 are ideal choices

In [22]:
rf5 = RandomForestRegressor(n_estimators=20,max_depth=5)
rf15 = RandomForestRegressor(n_estimators=20,max_depth=15)

In [23]:
rf5.fit(x_train,y_train)
r2_5 = rf5.score(x_test,y_test)
rf15.fit(x_train,y_train)
r2_15 = rf15.score(x_test,y_test)

RandomForestRegressor(bootstrap=True, criterion='mse', max_depth=5,
           max_features='auto', max_leaf_nodes=None,
           min_impurity_split=1e-07, min_samples_leaf=1,
           min_samples_split=2, min_weight_fraction_leaf=0.0,
           n_estimators=20, n_jobs=1, oob_score=False, random_state=None,
           verbose=0, warm_start=False)

RandomForestRegressor(bootstrap=True, criterion='mse', max_depth=15,
           max_features='auto', max_leaf_nodes=None,
           min_impurity_split=1e-07, min_samples_leaf=1,
           min_samples_split=2, min_weight_fraction_leaf=0.0,
           n_estimators=20, n_jobs=1, oob_score=False, random_state=None,
           verbose=0, warm_start=False)

In [24]:
print('r2 for random forest with max_depth=5 is %f'%r2_5)
print('r2 for random forest with max_depth=15 is %f'%r2_15)

r2 for random forest with max_depth=5 is 0.943212
r2 for random forest with max_depth=15 is 0.938540


In [25]:
def rf(ticker,window_=10):
    rf = RandomForestRegressor(max_depth=5)
    dat_rf = get_data(ticker,window_,21,'2009-01-01')
    y_train,x_train,y_test,x_test = split.train_test_split(dat_rf,'2016-01-01')
    rf.fit(x_train,y_train)
    r2 = rf.score(x_test,y_test)
    return r2

### Run RNN

In [48]:
def rnn(ticker,window_=10):
    dat_rf = get_data(ticker,window_,21,'2009-01-01')
    y_train,x_train,y_test,x_test = split.train_test_split(dat_rf,'2016-01-01')
    xtr = np.array(x_train)
    ytr = np.array(y_train)
    xte = np.array(x_test)
    yte = np.array(y_test)
    xtr_shape = xtr.shape
    xte_shape = xte.shape
    xtr = xtr.reshape(xtr_shape[0],window_,2)
    xte = xte.reshape(xte_shape[0],window_,2)
    from keras.models import Sequential
    from keras.layers import Dense,LSTM
    from keras.losses import mean_squared_error
    lstm = Sequential()
    lstm.add(LSTM(32,input_shape = (window_,2)))
    lstm.add(Dense(32,activation = 'relu'))
    lstm.add(Dense(16,activation = 'relu'))
    lstm.add(Dense(1,activation='relu'))
    lstm.compile(loss = mean_squared_error,optimizer='adam')
    lstm.fit(xtr,ytr,batch_size=15,epochs=10)
    lstm_result = lstm.predict(xte,batch_size=1).reshape(yte.shape[0],)
    r2_lstm = r2_score(yte,lstm_result)
    return r2_lstm

In [49]:
rnn('AAPL')

Epoch 1/10
1762/1762 [==============================] - 5s - loss: 4.8690        

0.91566714876734301